In [ ]:
import pickle
import numpy as np
from bff_processor.plotting_utils import meta_df, make_stack, plot_w_error
from bff_processor.utils import make_view, ratio_plot_template, nratio_plot_template
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import mplhep as hep
hep.set_style(hep.style.CMS)
import pickle
import pandas as pd

In [ ]:
lumi=35.5
pkl_name = '2016_reg_dict_DiLepMass_105'
with open(pkl_name+'.pkl','rb') as f:
    reg_dict_temp = pickle.load(f)

In [ ]:
region_count_dict = {}
for key in reg_dict_temp:
    snom, _, _, bin_center, color, label = make_stack(meta_df[meta_df.type=='background'],reg_dict_temp, key, sum_hist=False)
    region_count_dict[key] = {l:np.sum(x) for l,x in zip(label,snom)}
    print([np.sum(x) for x in snom])

In [ ]:
pd.DataFrame(region_count_dict).to_

In [ ]:
reg = 'SR1_{}'
snom, _, _, bin_center, color, label = make_stack(meta_df[meta_df.type=='background'],reg_dict_temp, reg, sum_hist=False)
SR1 = [np.sum(x) for x in snom]

In [ ]:
reg = 'SR2_{}'
snom, _, _, bin_center, color, label = make_stack(meta_df[meta_df.type=='background'],reg_dict_temp, reg, sum_hist=False)
SR2 = [np.sum(x) for x in snom]

In [ ]:
reg = 'CR13_{}'
snom, _, _, bin_center, color, label = make_stack(meta_df[meta_df.type=='background'],reg_dict_temp, reg, sum_hist=False)
CR13 = [np.sum(x) for x in snom]

In [ ]:
reg = 'CR23_{}'
snom, _, _, bin_center, color, label = make_stack(meta_df[meta_df.type=='background'],reg_dict_temp, reg, sum_hist=False)
CR23 = [np.sum(x) for x in snom]

In [ ]:
print('{} & {} & {} & {} & {}\\\\'.format('','mumu_1b','mumu_1b1j','ee_1b', 'ee_1b1j'))
for x in zip(label,SR1,SR2,CR13,CR23):
    print('{} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\'.format(*x))

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from bff_processor.SampleManager import SampleManager, SampleManagerPlotting
from bff_processor.SampleStack import SampleStack
from bff_processor.utils import make_view
import matplotlib.pyplot as plt
import mplhep as hep
import pprint
import matplotlib.gridspec as gridspec
hep.set_style(hep.style.CMS)
from uncertainties import ufloat

In [ ]:
era = 2016

In [ ]:
sample_list =[
   
['data/tw_{}_ZToEE_M_120_200.csv'.format(era), 'ZToEE_M_120_200', 'DY', 'bkc', 'red'],
['data/tw_{}_ZToEE_M_200_400.csv'.format(era), 'ZToEE_M_200_400', 'DY', 'bkc', 'red'],
['data/tw_{}_ZToEE_M_400_800.csv'.format(era), 'ZToEE_M_400_800', 'DY', 'bkc', 'red'],
['data/tw_{}_ZToEE_M_50_120.csv'.format(era), 'ZToEE_M_50_120', 'DY', 'bkc', 'red'],
['data/tw_{}_ZToEE_M_800_1400.csv'.format(era), 'ZToEE_M_800_1400', 'DY', 'bkc', 'red'],
['data/tw_{}_ZToMuMu_M_120_200.csv'.format(era), 'ZToMuMu_M_120_200', 'DY', 'bkc', 'red'],
['data/tw_{}_ZToMuMu_M_200_400.csv'.format(era), 'ZToMuMu_M_200_400', 'DY', 'bkc', 'red'],
['data/tw_{}_ZToMuMu_M_400_800.csv'.format(era), 'ZToMuMu_M_400_800', 'DY', 'bkc', 'red'],
['data/tw_{}_ZToMuMu_M_50_120.csv'.format(era), 'ZToMuMu_M_50_120', 'DY', 'bkc', 'red'],
['data/tw_{}_ZToMuMu_M_800_1400.csv'.format(era), 'ZToMuMu_M_800_1400', 'DY', 'bkc', 'red'],
    
['data/tw_{}_mc_santitop.csv'.format(era), 'mc_santitop', 'ST', 'bkc', 'green'],
['data/tw_{}_mc_stop.csv'.format(era), 'mc_stop', 'ST', 'bkc', 'green'],
['data/tw_{}_mc_ttbar.csv'.format(era), 'mc_ttbar', 'TT', 'bkc', 'orange'],
['data/tw_{}_mc_ww.csv'.format(era), 'mc_ww', 'WW/WZ/ZZ', 'bkc', 'purple'],
['data/tw_{}_mc_wz.csv'.format(era), 'mc_wz', 'WW/WZ/ZZ', 'bkc', 'purple'],
['data/tw_{}_mc_zz.csv'.format(era), 'mc_zz', 'WW/WZ/ZZ', 'bkc', 'purple']]
    


In [ ]:
stack = SampleStack(sample_list=sample_list)

In [ ]:
weights = [
['Weight_PuUp','Weight_PuDown'],
['Weight_BTagUp','Weight_BTagDown'],
['Weight_PUIDUp','Weight_PUIDDown'],
['Weight_PDF_ISRFSR_Up','Weight_PDF_ISRFSR_Down'],
['Weight_MuonSFUp','Weight_MuonSFDown'],
['Weight_ElectronSFUp','Weight_ElectronSFDown'],]
regions = ['SR{}_{{}}','CR{}0_{{}}', 'CR{}3_{{}}', 'CR{}4_{{}}']
nJets = [1,2]
jes_var = [['jesDown', 'jesUp'], ['jerDown', 'jerUp']]
BFF_cuts =  ['HTLT_{}', 'RelMET_{}', 'SBM_{}']
def def_BFF_cuts(jv, nJet):
    t1 =  [bff.format(jv) for bff in BFF_cuts]
    return(list(zip(t1,BFF_cut_values[nJet])))

BFF_cut_values ={1:[-18,.3125,18.75], 2:[-10,.2625,1.25]}
def make_view(jv,nJets, region): 
    bff = def_BFF_cuts(jv, nJets)
    view_dict = {'lt': {'DiLepMass': np.inf, bff[0][0]: bff[0][1], bff[1][0]: bff[1][1]},
         'gt': {'DiLepMass': -np.inf, bff[2][0]: bff[2][1]},
         'eq': { region.format(nJets).format(jv): 1},
         'eq': {'Flag_METFilters': 1, region.format(nJets).format(jv): 1}}
    return view_dict

In [ ]:

view = make_view('nominal',1,'CR{}0_{{}}')
stack.view = view

for smp in stack.SMP():
    print(smp.name, smp.total(sumW2=1))
#nom_hist = stack.sum_hists(column, bins, w_kwargs={'weight_names':['Weight']},label='DY')

In [ ]:
region_count_dict = {}
for key in reg_dict_temp:
    nJets=1
    key= (key.format('{{}}'))
    if key[2]=='2': nJets=2
    view = make_view('nominal',nJets,key)
    print(view)
    stack.view = view
    region_count_dict[key] = {smp.name: ufloat(*smp.total(sumW2=1)) for smp in stack.SMP()}
    #print([np.sum(x) for x in snom])

In [ ]:
df = pd.DataFrame(region_count_dict)

In [ ]:
df

In [ ]:
import re
columns = df.columns
onejet = [c for c in columns if re.match('(SR|CR)1[0-9]*',c)]
twojet = [c for c in columns if re.match('(SR|CR)2[0-9]*',c)]
onejet = ['SR1_{{}}',  'CR13_{{}}',  'CR10_{{}}','CR14_{{}}']
twojet = ['SR2_{{}}', 'CR23_{{}}', 'CR20_{{}}',  'CR24_{{}}']

dfone = df[onejet]
dtwo = df[twojet]

In [ ]:
twojet

In [ ]:
dfone #re.findall('((?:SR|CR)2[0-9]*)',c)[0]

In [ ]:
def reduce_df(df):
    dy_sum = df.iloc[0:10].sum(axis=0)
    dy_sum.name='mc_DY'
    df = df.append(dy_sum)
    df.drop(index=df.iloc[0:10, :].index.tolist(), inplace=True)
    dy_sum = df.iloc[0:2].sum(axis=0)
    dy_sum.name='mc_ST'
    df = df.append(dy_sum)
    df.drop(index=df.iloc[0:2, :].index.tolist(), inplace=True)
    return df

In [ ]:
dfone = reduce_df(dfone)
dtwo = reduce_df(dtwo)

In [ ]:
dfone.sort_values(by=['SR1_{{}}'])

In [ ]:
dfone.divide(dfone.sum(axis=0)).sort_values(by=['SR1_{{}}'])

In [ ]:
print(dfone.sort_values(by=['SR1_{{}}']).to_latex())

In [ ]:
print((dfone.divide(dfone.sum(axis=0))*100).round(1).sort_values(by=['SR1_{{}}']).to_latex())

In [ ]:
print(dtwo.sort_values(by=['SR2_{{}}']).apply(lambda x : x.map(lambda y: y.nominal_value), axis=1).round(1).to_latex())
print((dtwo.divide(dtwo.sum(axis=0))*100).round(1).sort_values(by=['SR2_{{}}']).to_latex())

In [ ]:
print(dfone.sort_values(by=['SR1_{{}}']).apply(lambda x : x.map(lambda y: y.nominal_value), axis=1).round(1).to_latex())
print((dfone.divide(dfone.sum(axis=0))*100).round(1).sort_values(by=['SR1_{{}}']).to_latex())